In [ ]:
## import tensorflow as tf
import keras
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, BatchNormalization, Dropout
from keras import regularizers
from keras.activations import relu
import numpy as np
import cv2 as cv
from sklearn.model_selection import train_test_split
import os
import random

face_cascade = cv.CascadeClassifier(cv.data.haarcascades + 'haarcascade_frontalface_default.xml')

def detect_and_crop_face(image):
    # Detect face in the grayscale image
    faces = face_cascade.detectMultiScale(image, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    cropped_faces = []
    for (x, y, w, h) in faces:
        cropped_faces.append(image[y:y+h, x:x+w])
    return cropped_faces
#     if len(faces) > 0:
#         (x, y, w, h) = faces[0]
#         face = image[y:y+h, x:x+w]
#         return [face]
#     return []

dataset = '.\lfw'
images = []
labels = []
label_map = {} # maps names to indexes since numerical labels are required

for folder_name in os.listdir(dataset):
    # the folder name corresponds to the person's name
    if folder_name not in label_map:
        label_map[folder_name] = len(label_map)
    label_idx = label_map[folder_name]
    
    for image_name in os.listdir(os.path.join(dataset, folder_name)):
        image_path = os.path.join(dataset, folder_name, image_name)
        image = cv.imread(image_path)
        grayscaled_image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)       
        resized_image = cv.resize(grayscaled_image, (128, 128))
        cropped_images = detect_and_crop_face(resized_image)
        
        for img in cropped_images:
#             cv.imwrite(os.path.join('/users/jerome akumasi/desktop/cropped', f'{image_name}_{random.randint(0, 100000)}.jpg'), img)
            images.append(img)
            labels.append(label_idx)
        
            
#         equalized_image = cv.equalizeHist(cropped_image)
        
#         images.append(resized_image)
#         labels.append(label_idx)
        
print(images[0])
    
# images = np.array(images, dtype=np.float32)        
        
x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size=0.2)
x_train = np.array(x_train, dtype=np.float32) / 255.0 # normalize
# y_train = np.array(y_train, dtype=np.int32)
# x_test  = np.array(x_test, dtype=np.float32) / 255.0  # normalize
# y_test  = np.array(y_test, dtype=np.int32)



def cnn_model():
    num_of_classes = len(label_map)
    
    inputs = keras.Input(shape=(128, 128, 1))
    x = Conv2D(10, (2,2), padding='same', kernel_regularizer=regularizers.l2())(inputs) # conv 1
    x = BatchNormalization()(x) # norm 1
    x = relu(x) # activation 1
    x = MaxPooling2D()(x) # pooling 1
    
    x = Conv2D(20, (2,2), padding='same', kernel_regularizer=regularizers.l2())(x) # conv 2
    x = BatchNormalization()(x) # norm 2
    x = relu(x) # activation 2
    x = MaxPooling2D()(x) # pooling 2
    
    x = Conv2D(30, (2,2), padding='same', kernel_regularizer=regularizers.l2())(x) # conv 3
    x = BatchNormalization()(x) # norm 3
    x = relu(x) # activation 3
    x = MaxPooling2D()(x) # pooling 3
    
    x = Conv2D(40, (2,2), padding='same', kernel_regularizer=regularizers.l2())(x) # conv 4
    x = BatchNormalization()(x) # norm 4
    x = relu(x) # activation 4
    x = MaxPooling2D()(x) # pooling 4
    
    x = Flatten()(x) # flatten
    x = Dense(64, activation='relu')(x) # dense 1
    x = Dropout(0.5)(x) # 50% dropout
    outputs = Dense(num_of_classes, activation='softmax')(x) # dense 2, outputs
    
    model = keras.Model(inputs=inputs, outputs=outputs) 
    return model


model = cnn_model()

loss = keras.losses.SparseCategoricalCrossentropy()
optimizer = keras.optimizers.Adam()
model.compile(
    loss=loss, 
    optimizer=optimizer, 
    metrics=['accuracy']
)

epochs = 10
# model.fit(x_train, y_train, epochs=epochs, verbose=2)

# model.evaluate(x_test, y_test, batch_size=32, verbose=2)

# keras.models.save_model(model, './model/recog.h5')


    